# Social Graphs 2021 - Assignment 2
## Quick disclaimer
The assignment description could hint at not implementing some of the previous exercises, but just use the files already downloaded and processed. This seemed to make the exercises and explanations a little incoherrent, so we have included the code for the API-calls and other misc stuff. We have tried to collapse the cell blocks, so this is not annoying for the reader, but if they do appear, sorry for the inconvinience.

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wordcloud
import nltk
import networkx as nx

import re
import json
import urllib

import os

In [2]:
# Styles and data. If this cell block is not already hidden/minimized, feel free to do so, as it only contains the styles for the plots, 
# and the characters.csv in written form, as it allows for reruns of the code, if needed.
DTU_COLORS = dict(
    dtured      = "#990000",
    blue        = "#2F3EEA",
    brightgreen = "#1FD082",
    navyblue    = "#030F4F",
    
)

# 0. Building the network
## Strategy for extracting hyperlinks from the wiki-pages
The strategy for extracting the hyperlinks is to make 3 webcalls for allies, enemies and bosses. For enemies as a single case, we remove the traps part of the html-string, as it is not included. From here we use regex expressions explained in the next part to extract each character name, which we will use to directly query the API, for each of the characters.

In [3]:
# Gets the json format of the 3 pages with characters (allies, enemies and bosses)
def get_json(title):
    baseurl = "https://zelda.fandom.com/api.php?"
    action = "action=query"
    title = "titles={}".format(title)
    content = "prop=revisions&rvprop=content&rvslots=*"
    dataformat ="format=json"

    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)

    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')

    return json.loads(wikitext)

# Extracts the content of the pages, yielding the title of the page, and the html-string 
# with character names.
def displayWiki(wiki):
    code = str(list(wiki["query"]["pages"].keys())[0])
    title = wiki["query"]["pages"][code]["title"]
    content = wiki["query"]["pages"][code]["revisions"][0]["slots"]["main"]["*"]
    return title, content

allies_json = get_json("Characters_in_Breath_of_the_Wild")
enemies_json = get_json("Enemies_in_Breath_of_the_Wild")
bosses_json = get_json("Bosses_in_Breath_of_the_Wild")

allies_title, allies_content = displayWiki(allies_json)
enemies_title, enemies_content = displayWiki(enemies_json)
bosses_title, bosses_content = displayWiki(bosses_json)
enemies_content = enemies_content.split("Traps")[0]

# Regex strategy
We do two different regex calls. First we find the characters in the 

The main regular expression is to find every link o a character. We know they have the shape `{{Term|BotW|name of character|link|maybe more stuff}}`. Unfortionately in the texts locations follow the same pattern. This is first removed with the expression `{{Small.*?'}}`, to filter out the locations.

In [4]:
find_expr = r"{{Term\|BotW\|(.*?)\|link[\|]*.*?}}"
delete_locations = r"{{Small.*?'}}"

bosses_content = re.sub(delete_locations, "", bosses_content)
allies = re.findall(find_expr, allies_content)
enemies = re.findall(find_expr, enemies_content)
bosses = re.findall(find_expr, bosses_content)